# GOLDILOX Insights - Permission Setup

This notebook helps you configure the permissions that **GOLDILOX Insights** needs to analyze your query patterns and generate clustering recommendations.

**Run this notebook as ACCOUNTADMIN** (or a role with sufficient privileges).

### Permissions covered in this notebook

| Permission | Purpose |
|---|---|
| **MONITOR on warehouses** | Allows the app to fetch query profiles and analyze warehouse performance |
| **Database role grants (USAGE + SELECT)** | Allows the app to read table metadata and query patterns from your monitored databases |

> These permissions **cannot** be granted from within the app itself. The app will detect missing permissions and show warnings, but an admin must run these grants manually.

### Before you begin

1. Make sure the GOLDILOX Insights app is already installed
2. Identify which **warehouses** and **databases** you want the app to monitor

> **App name:** This notebook assumes the default app name `GOLDILOX_INSIGHTS`. If you renamed the app during installation, update the `APP_NAME` variable in the Configuration cell below.

## Configuration

The default app name is `GOLDILOX_INSIGHTS`. If you renamed the app during installation, update the value below. Otherwise, run this cell as-is.

In [ ]:
-- Default app name (update only if you renamed the app during installation)
SET APP_NAME = 'GOLDILOX_INSIGHTS';
SET MISSING_PERMISSIONS_VIEW = $APP_NAME || '.SHARED_VIEWS.MISSING_PERMISSIONS';
SELECT $APP_NAME AS APP_NAME, $MISSING_PERMISSIONS_VIEW AS MISSING_PERMISSIONS_VIEW;

---
## 1. Grant MONITOR Privilege on Warehouses

The app needs **MONITOR** privilege on each warehouse you want it to analyze. This allows the app to:
- Fetch query profiles for queries executed on that warehouse
- Analyze warehouse performance and partition pruning patterns
- Track query execution metrics over time

Without MONITOR, the app can still see basic query history (via `ACCOUNT_USAGE`), but **cannot** retrieve detailed query profiles needed for accurate clustering recommendations.

### Step 1a: Discover your active warehouses

Run the cell below to see all warehouses with recent query activity.

In [ ]:
-- List active warehouses with query counts from the last 30 days
SELECT
    WAREHOUSE_NAME,
    COUNT(*) AS QUERY_COUNT_30D,
    SUM(PARTITIONS_SCANNED) AS TOTAL_PARTITIONS_SCANNED,
    SUM(PARTITIONS_TOTAL) AS TOTAL_PARTITIONS_TOTAL
FROM SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY
WHERE
    QUERY_TYPE = 'SELECT'
    AND WAREHOUSE_SIZE IS NOT NULL
    AND DATABASE_NAME != 'SNOWFLAKE'
    AND START_TIME >= DATEADD(DAY, -30, CURRENT_DATE())
GROUP BY WAREHOUSE_NAME
ORDER BY QUERY_COUNT_30D DESC;

### Step 1b: Check existing MONITOR grants

Run this to see which warehouses already have MONITOR granted to the app.

In [ ]:
-- Check which warehouses already have MONITOR granted to the app
SELECT
    NAME AS WAREHOUSE_NAME,
    PRIVILEGE,
    GRANTED_ON,
    GRANTED_BY
FROM SNOWFLAKE.ACCOUNT_USAGE.GRANTS_TO_ROLES
WHERE
    GRANTEE_NAME = $APP_NAME
    AND GRANTED_ON = 'WAREHOUSE'
    AND DELETED_ON IS NULL
ORDER BY NAME;

### Step 1c: Grant MONITOR on warehouses

**Option A:** Run the cell below to generate GRANT statements based on warehouses the app has detected as missing MONITOR access. Review the output and run the generated SQL in the cell after it.

> **Note:** This requires the app to have run at least once so it can detect which warehouses need access. If the app hasn't run yet, use Option B (manual grant) instead.

In [ ]:
-- Generate GRANT MONITOR statements for warehouses missing permissions
-- Review the output, then copy and run in the next cell
SELECT
    'GRANT MONITOR ON WAREHOUSE ' || OBJECT_NAME || ' TO APPLICATION ' || $APP_NAME || ';' AS GRANT_SQL
FROM IDENTIFIER($MISSING_PERMISSIONS_VIEW)
WHERE OBJECT_TYPE = 'WAREHOUSE'
AND PERMISSION_TYPE = 'MONITOR'
ORDER BY GRANT_SQL;

In [ ]:
-- Paste the generated GRANT statements from above and run


**Option B:** Manually grant MONITOR on specific warehouses. Update the warehouse names below to match your environment.

In [ ]:
-- Manual grant: update warehouse names to match your environment

GRANT MONITOR ON WAREHOUSE COMPUTE_WH    TO APPLICATION IDENTIFIER($APP_NAME);
-- GRANT MONITOR ON WAREHOUSE ANALYTICS_WH  TO APPLICATION IDENTIFIER($APP_NAME);
-- GRANT MONITOR ON WAREHOUSE ETL_WH         TO APPLICATION IDENTIFIER($APP_NAME);

### Step 1d: (Optional) Auto-grant via scheduled task

If you want missing MONITOR permissions to be granted automatically, create a scheduled task that runs daily. This task checks the app's missing permissions view and grants MONITOR on any warehouses that need it.

> **Important:**
> - This task runs as the role that creates it (typically ACCOUNTADMIN). Make sure you are comfortable with automatic grants before enabling this.
> - Update the **warehouse name** (`GOLDILOX_APP_TASK_WAREHOUSE`) to a warehouse available in your environment.
> - If you renamed the app during installation, update the two references to `GOLDILOX_INSIGHTS` in the task body below (the FROM clause and the GRANT statement). Session variables do not persist inside task bodies, so these must be hardcoded.

In [ ]:
-- Create a scheduled task to auto-grant MONITOR on warehouses missing permissions
-- >>> Update GOLDILOX_APP_TASK_WAREHOUSE to a warehouse available in your environment <<<

CREATE SCHEMA IF NOT EXISTS GOLDILOX_INSIGHTS_CLIENT_WORKSPACE.TASKS;

CREATE OR REPLACE TASK GOLDILOX_INSIGHTS_CLIENT_WORKSPACE.TASKS.AUTO_GRANT_WAREHOUSE_MONITOR
  WAREHOUSE = GOLDILOX_APP_TASK_WAREHOUSE
  SCHEDULE = 'USING CRON 0 7 * * * UTC'
  COMMENT = 'Auto-grant MONITOR on warehouses detected by GOLDILOX Insights'
AS
DECLARE
  grant_sql VARCHAR;
  c1 CURSOR FOR
    SELECT OBJECT_NAME
    FROM GOLDILOX_INSIGHTS.SHARED_VIEWS.MISSING_PERMISSIONS
    WHERE OBJECT_TYPE = 'WAREHOUSE'
    AND PERMISSION_TYPE = 'MONITOR';
BEGIN
  FOR rec IN c1 DO
    grant_sql := 'GRANT MONITOR ON WAREHOUSE ' || rec.OBJECT_NAME || ' TO APPLICATION GOLDILOX_INSIGHTS';
    EXECUTE IMMEDIATE grant_sql;
  END FOR;
END;

-- Activate And Run the task immediately
ALTER TASK GOLDILOX_INSIGHTS_CLIENT_WORKSPACE.TASKS.AUTO_GRANT_WAREHOUSE_MONITOR RESUME;
EXECUTE TASK GOLDILOX_INSIGHTS_CLIENT_WORKSPACE.TASKS.AUTO_GRANT_WAREHOUSE_MONITOR;

---
## 2. Grant Database Access for Monitored Tables

The app needs **USAGE** and **SELECT** access on databases containing tables you want to monitor. This is done by creating a database role (`GOLDILOX_APP_VIEWER`) in each target database and granting it to the app.

This allows the app to:
- Read table metadata (row counts, clustering keys, storage sizes)
- Analyze column usage patterns for clustering recommendations
- Validate access status for monitored targets

### Step 2a: Identify databases to monitor

Run this to see databases with the most query activity and partition scanning.

In [ ]:
-- List databases with significant query activity in the last 30 days
SELECT
    DATABASE_NAME,
    COUNT(*) AS QUERY_COUNT_30D,
    SUM(PARTITIONS_SCANNED) AS TOTAL_PARTITIONS_SCANNED,
    SUM(PARTITIONS_TOTAL) AS TOTAL_PARTITIONS_TOTAL,
    CASE
        WHEN SUM(PARTITIONS_TOTAL) = 0 THEN 0
        ELSE ROUND((SUM(PARTITIONS_SCANNED) / SUM(PARTITIONS_TOTAL)) * 100, 2)
    END AS SCAN_PERCENTAGE
FROM SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY
WHERE
    QUERY_TYPE = 'SELECT'
    AND WAREHOUSE_SIZE IS NOT NULL
    AND DATABASE_NAME IS NOT NULL
    AND DATABASE_NAME != 'SNOWFLAKE'
    AND START_TIME >= DATEADD(DAY, -30, CURRENT_DATE())
GROUP BY DATABASE_NAME
ORDER BY TOTAL_PARTITIONS_SCANNED DESC;

### Step 2b: Check existing database grants

Run this to see which databases already have the `GOLDILOX_APP_VIEWER` role granted.

In [ ]:
-- Check which databases already have GOLDILOX_APP_VIEWER role granted to the app
SELECT
    NAME,
    PRIVILEGE,
    GRANTED_ON,
    GRANTED_BY
FROM SNOWFLAKE.ACCOUNT_USAGE.GRANTS_TO_ROLES
WHERE
    GRANTEE_NAME LIKE '%GOLDILOX_APP_VIEWER%'
    AND DELETED_ON IS NULL
ORDER BY NAME;

### Step 2c: Grant access for each database

For each database you want to monitor, run a grant block like the one below. This creates a `GOLDILOX_APP_VIEWER` database role, grants it to the app, and gives it USAGE and SELECT access.

**Copy and modify this block for each database.** Replace `MY_DATABASE` with your actual database name.

> **Note on quoting:** If your database name contains mixed case or special characters, keep the double quotes (e.g., `"My_Database"`). For all-uppercase names, quotes are optional but harmless.

In [ ]:
-- ================================================
-- Grant access for database: MY_DATABASE
-- Replace MY_DATABASE with your actual database name
-- ================================================

-- Create the database role
CREATE DATABASE ROLE IF NOT EXISTS "MY_DATABASE".GOLDILOX_APP_VIEWER;

-- Grant the role to the app
GRANT DATABASE ROLE "MY_DATABASE".GOLDILOX_APP_VIEWER TO APPLICATION IDENTIFIER($APP_NAME);

-- Grant USAGE on the database and all schemas
GRANT USAGE ON DATABASE "MY_DATABASE" TO DATABASE ROLE "MY_DATABASE".GOLDILOX_APP_VIEWER;
GRANT USAGE ON ALL SCHEMAS IN DATABASE "MY_DATABASE" TO DATABASE ROLE "MY_DATABASE".GOLDILOX_APP_VIEWER;
GRANT USAGE ON FUTURE SCHEMAS IN DATABASE "MY_DATABASE" TO DATABASE ROLE "MY_DATABASE".GOLDILOX_APP_VIEWER;

-- Grant SELECT on all current and future tables
GRANT SELECT ON ALL TABLES IN DATABASE "MY_DATABASE" TO DATABASE ROLE "MY_DATABASE".GOLDILOX_APP_VIEWER;
GRANT SELECT ON FUTURE TABLES IN DATABASE "MY_DATABASE" TO DATABASE ROLE "MY_DATABASE".GOLDILOX_APP_VIEWER;

Duplicate the cell above for additional databases. Uncomment and update the example below, or add new SQL cells as needed.

In [ ]:
-- ================================================
-- Grant access for database: SECOND_DATABASE
-- Uncomment and update the database name
-- ================================================

-- CREATE DATABASE ROLE IF NOT EXISTS "SECOND_DATABASE".GOLDILOX_APP_VIEWER;
-- GRANT DATABASE ROLE "SECOND_DATABASE".GOLDILOX_APP_VIEWER TO APPLICATION IDENTIFIER($APP_NAME);
-- GRANT USAGE ON DATABASE "SECOND_DATABASE" TO DATABASE ROLE "SECOND_DATABASE".GOLDILOX_APP_VIEWER;
-- GRANT USAGE ON ALL SCHEMAS IN DATABASE "SECOND_DATABASE" TO DATABASE ROLE "SECOND_DATABASE".GOLDILOX_APP_VIEWER;
-- GRANT USAGE ON FUTURE SCHEMAS IN DATABASE "SECOND_DATABASE" TO DATABASE ROLE "SECOND_DATABASE".GOLDILOX_APP_VIEWER;
-- GRANT SELECT ON ALL TABLES IN DATABASE "SECOND_DATABASE" TO DATABASE ROLE "SECOND_DATABASE".GOLDILOX_APP_VIEWER;
-- GRANT SELECT ON FUTURE TABLES IN DATABASE "SECOND_DATABASE" TO DATABASE ROLE "SECOND_DATABASE".GOLDILOX_APP_VIEWER;

---
## 3. Verify Permissions

After granting permissions, run the cells below to confirm everything is in place.

### Verify warehouse MONITOR grants

In [ ]:
-- Verify MONITOR grants on warehouses for the app
SELECT
    NAME AS WAREHOUSE_NAME,
    PRIVILEGE
FROM SNOWFLAKE.ACCOUNT_USAGE.GRANTS_TO_ROLES
WHERE
    GRANTEE_NAME = $APP_NAME
    AND GRANTED_ON = 'WAREHOUSE'
    AND PRIVILEGE = 'MONITOR'
    AND DELETED_ON IS NULL
ORDER BY NAME;

### Verify database access grants

In [ ]:
-- Verify database role grants for GOLDILOX_APP_VIEWER
SELECT
    NAME,
    PRIVILEGE,
    GRANTED_ON,
    TABLE_CATALOG AS DATABASE_NAME
FROM SNOWFLAKE.ACCOUNT_USAGE.GRANTS_TO_ROLES
WHERE
    GRANTEE_NAME LIKE '%GOLDILOX_APP_VIEWER%'
    AND DELETED_ON IS NULL
ORDER BY TABLE_CATALOG, GRANTED_ON, NAME;

---
## Next Steps

After running this notebook:

1. Go back to the **GOLDILOX Insights** app
2. Navigate to the **Recommendations** tab
3. Click **Refresh Access Status** to confirm the app detects the new permissions
4. Add your target databases/tables in the **Monitored Targets** section

### Adding more databases later

You can re-run this notebook anytime to grant access to additional databases or warehouses. Just add new grant blocks in Sections 1c and 2c above.

### Revoking access

> **Warning:** Revoking access will immediately stop the app from monitoring the affected warehouses or databases. Any in-progress analysis or recommendations for those resources will be interrupted. Please consult the Goldilox team at **support@goldilox.com** before revoking access.

To revoke the app's access to a database:
```sql
REVOKE DATABASE ROLE "MY_DATABASE".GOLDILOX_APP_VIEWER FROM APPLICATION IDENTIFIER($APP_NAME);
DROP DATABASE ROLE IF EXISTS "MY_DATABASE".GOLDILOX_APP_VIEWER;
```

To revoke warehouse monitoring:
```sql
REVOKE MONITOR ON WAREHOUSE MY_WAREHOUSE FROM APPLICATION IDENTIFIER($APP_NAME);
```